In [1]:
# !pip install sentencepiece

In [2]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
import pandas as pd
from tqdm import tqdm

In [68]:
import math

def rescore(text: str, tokenizer: T5Tokenizer,
            model: T5ForConditionalGeneration) -> str:
    
    if isinstance(text, float) and math.isnan(text): # if an input text is empty, then we return an empty text too
        return ''
    elif len(text) == 0:  # if an input text is empty, then we return an empty text too
        return ''
    ru_letters = set('аоуыэяеёюибвгдйжзклмнпрстфхцчшщьъ')
    punct = set('.,:/\\?!()[]{};"\'-')
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    min_size = 3
    if x.input_ids.shape[1] <= min_size:
        return text  # we don't rescore a very short text
    out = model.generate(**x, do_sample=False, num_beams=5,
                         max_length=max_size, min_length=min_size)
    res = tokenizer.decode(out[0], skip_special_tokens=True).lower().strip()
    res = ' '.join(res.split())
    postprocessed = ''
    for cur in res:
        if cur.isspace() or (cur in punct):
            postprocessed += ' '
        elif cur in ru_letters:
            postprocessed += cur
    return (' '.join(postprocessed.strip().split())).replace('ё', 'е')

In [4]:
model_name = 'bond005/ruT5-ASR'

tokenizer_for_rescoring = T5Tokenizer.from_pretrained(model_name)
model_for_rescoring = T5ForConditionalGeneration.from_pretrained(model_name)

if torch.cuda.is_available():
    model_for_rescoring = model_for_rescoring.cuda()

In [13]:
train_df = pd.read_csv('train_df.csv')
test_df = pd.read_csv('test_df.csv')

train_df.head(1)

,transcription,gt
0,демократия неумально подвегается пафу и арабск...,"Демократия неумолимо продвигается по Африке, и..."


In [26]:
rescored_results = []

for i in tqdm(train_df["transcription"]):
    rescored = rescore(i, tokenizer_for_rescoring, model_for_rescoring)
    rescored_results.append(rescored)
    
train_df["rescored_transcription"] = rescored_results

100%|██████████| 22862/22862 [1:12:36<00:00,  5.25it/s]


In [69]:
test_rescored_results = []

for i in tqdm(test_df["transcription"]):
    rescored = rescore(i, tokenizer_for_rescoring, model_for_rescoring)
    test_rescored_results.append(rescored)
    
test_df["rescored_transcription"] = test_rescored_results

100%|██████████| 9630/9630 [30:54<00:00,  5.19it/s]


In [70]:
train_df.to_csv('train_df_rescored.csv', index=False)
test_df.to_csv('test_df_rescored.csv', index=False)

In [25]:
rescored_results

['демократия неумолимо подстегивает пауэлла и арабская весна была ее концом',
 'доклад международного одета папа нагиш',
 'мы разъясняем ему их права и законы',
 'только что завершившееся председательствование группы демократии',
 'ведь буфет до последнего человека разом превратился в зал',
 'в развитии континента достигнут зачислительный прогресс',
 'это позволит сберечь еще больше жизней в предстоящие годы',
 'бангладеш каждый год терует голову ну прямо вот чем национальный день кстати',
 'мы должны положить глаз на место для развития на основе сотрудничества',
 'как руководители стран мира вы должны остановиться задуматься и дать гражданам отдохнуть',
 'слово имеет посол эквадора альпонсо моравьев',
 'вот это уж наш палев',
 'ты не можешь судить только девица не может тать и престить войси',
 'такую принципиальную позицию занимает государство члены этого органа']